## Ollama 
* genration and chat completion
    * [Ollama  completion refrence](https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-chat-completion).

In [6]:
import requests
import json

# Define the API URL for the Ollama generation endpoint
generation_api_url = "http://localhost:11434/api/generate"

# Specify the model to be used for text generation
model = "llama3.2:latest"

# Define the input text to be summarized
text = "Hello, how are you?"

# Send a POST request to the Ollama API with the required parameters
response = requests.post(
    generation_api_url,
    json={
        "model": model,  # Specify the model to use
        "prompt": f"Summarize the tone, intent, and structure of the following text in 3 bullet points: {text}",  # Define the task prompt
        "stream": False,  
        "options": {  
            "temperature": 0.7,  
            "top_p": 0.9,  
            "top_k": 50,  
            "repetition_penalty": 1.2  # Penalize repetition in the output
        }
    }
)
response.json()

{'model': 'llama3.2:latest',
 'created_at': '2025-05-02T15:11:08.327216473Z',
 'response': 'Here is a summary of the tone, intent, and structure of the given text:\n\n• **Tone:** The tone of this sentence is informal, friendly, and welcoming. It sets a casual and approachable tone.\n• **Intent:** The intent of this sentence is to initiate a conversation or establish a connection with the person being addressed. It\'s a greeting that invites the other person to respond.\n• **Structure:** The structure of this sentence is simple and straightforward. It consists of a single sentence with a short question ("how are you?") followed by a polite expression of inquiry.',
 'done': True,
 'done_reason': 'stop',
 'context': [128006,
  9125,
  128007,
  271,
  38766,
  1303,
  33025,
  2696,
  25,
  6790,
  220,
  2366,
  18,
  271,
  128009,
  128006,
  882,
  128007,
  271,
  9370,
  5730,
  553,
  279,
  16630,
  11,
  7537,
  11,
  323,
  6070,
  315,
  279,
  2768,
  1495,
  304,
  220,
  18,

In [8]:
response.json()['response']

'Here is a summary of the tone, intent, and structure of the given text:\n\n• **Tone:** The tone of this sentence is informal, friendly, and welcoming. It sets a casual and approachable tone.\n• **Intent:** The intent of this sentence is to initiate a conversation or establish a connection with the person being addressed. It\'s a greeting that invites the other person to respond.\n• **Structure:** The structure of this sentence is simple and straightforward. It consists of a single sentence with a short question ("how are you?") followed by a polite expression of inquiry.'

In [9]:
from pprint import pprint as pprint
pprint(response.json()['response'])

('Here is a summary of the tone, intent, and structure of the given text:\n'
 '\n'
 '• **Tone:** The tone of this sentence is informal, friendly, and welcoming. '
 'It sets a casual and approachable tone.\n'
 '• **Intent:** The intent of this sentence is to initiate a conversation or '
 "establish a connection with the person being addressed. It's a greeting that "
 'invites the other person to respond.\n'
 '• **Structure:** The structure of this sentence is simple and '
 'straightforward. It consists of a single sentence with a short question '
 '("how are you?") followed by a polite expression of inquiry.')


##### chat completion

In [13]:
import json
import requests

# Define the API URL for the Ollama chat endpoint
chat_api_url = "http://localhost:11434/api/chat"

# Specify the model to be used for chat
model = "llama3.2:latest"

# Define the input text for the chat
text = "Hello! what are the beautiful things in egypt?"

# Send a POST request to the Ollama API with the required parameters    
response = requests.post(
    chat_api_url,
    json={
        "model": model,  
        "messages": [
            {"role": "user",
            "content": text}  
        ],
        "stream": False,  
        "options": {  
            "temperature": 0.7,  
            "top_p": 0.9,  
            "top_k": 50,
            "seed" : 101,  
            "repetition_penalty": 1.2,
            "num_predict": 100,
        }
    }
)
response.json()

{'model': 'llama3.2:latest',
 'created_at': '2025-05-02T15:29:00.117193313Z',
 'message': {'role': 'assistant',
  'content': 'Egypt is a country rich in history, culture, and natural beauty. Here are some of the most beautiful things in Egypt:\n\n1. **Pyramids of Giza**: No list of Egyptian wonders would be complete without these iconic pyramids, one of the Seven Wonders of the Ancient World.\n2. **Nile River**: The lifeblood of ancient Egypt, the Nile is a stunning river that winds its way through the heart of the country, offering breathtaking views and opportunities for boat rides and fel'},
 'done_reason': 'length',
 'done': True,
 'total_duration': 5736191793,
 'load_duration': 23456757,
 'prompt_eval_count': 36,
 'prompt_eval_duration': 14679662,
 'eval_count': 100,
 'eval_duration': 5697555450}

In [14]:
from pprint import pprint as pprint

pprint(response.json()['message']['content'])

('Egypt is a country rich in history, culture, and natural beauty. Here are '
 'some of the most beautiful things in Egypt:\n'
 '\n'
 '1. **Pyramids of Giza**: No list of Egyptian wonders would be complete '
 'without these iconic pyramids, one of the Seven Wonders of the Ancient '
 'World.\n'
 '2. **Nile River**: The lifeblood of ancient Egypt, the Nile is a stunning '
 'river that winds its way through the heart of the country, offering '
 'breathtaking views and opportunities for boat rides and fel')


In [21]:
response = requests.post(
    chat_api_url,
    json={
        "model": model,
        "messages": [
            {"role": "user", "content": "Hello! what are the beautiful things in egypt?"},
            {"role": "assistant", "content": "Egypt is home to the Pyramids of Giza, the Nile River, and Luxor’s temples."},
            {"role": "user", "content": "Tell me more about the Nile!"}
        ],
        "stream": False,
        "options": {
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
            "seed": 101,
            "repetition_penalty": 1.2,
            "num_predict": 100,
            "num_ctx": 4096  # Allows more context for conversation history
        }
    }
)
print(response.json()['message']['content'])

The Nile River is an iconic and fascinating part of Egyptian culture and history. Here are some interesting facts about the Nile:

1. **Longest River in Africa**: The Nile is approximately 6,695 kilometers (4,160 miles) long, stretching from its sources in Burundi and Rwanda to its delta on the Mediterranean Sea.
2. **Lifeblood of Egypt**: The Nile has been the primary source of water, transportation, and food for millions of people in Egypt for thousands of years.



In [20]:
response.json()

{'model': 'llama3.2:latest',
 'created_at': '2025-05-02T16:50:17.958797887Z',
 'message': {'role': 'assistant',
  'content': 'The Nile River is an iconic and fascinating part of Egyptian culture and history. Here are some interesting facts about the Nile:\n\n1. **Longest River in Africa**: The Nile is approximately 6,695 kilometers (4,160 miles) long, stretching from its sources in Burundi and Rwanda to its delta on the Mediterranean Sea.\n2. **Lifeblood of Egypt**: The Nile has been the primary source of water, transportation, and food for millions of people in Egypt for thousands of years.\n'},
 'done_reason': 'length',
 'done': True,
 'total_duration': 8315765639,
 'load_duration': 2120388996,
 'prompt_eval_count': 74,
 'prompt_eval_duration': 362473346,
 'eval_count': 100,
 'eval_duration': 5831249827}

In [ ]:
import base64
from PIL import Image
import io

def encode_image_to_base64(image_path:str):
    with open(image_path, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def send_image_to_ollama(image_path:str):
    image_base64 = encode_image_to_base64(image_path)
    response = requests.post(
        chat_api_url,
        json={
            "model": model,
            "messages": [
                {
                    "role": "user", 
                    "content": "what is in this image?",
                    "images": [image_base64]
                }
            ],
            "stream": False,
            "options": {
                "temperature": 0.7,
                "top_p": 0.9,
                "top_k": 50,
                "seed": 101,
                "repetition_penalty": 1.2,
                "num_predict": 100
            }
        }
    )
    return response.json()

# Example usage
image_path = "path_to_your_image.jpg"  # Replace with your image path
response = send_image_to_ollama(image_path)
print(response['message']['content'])

### [OpenAI](https://github.com/ollama/ollama/blob/main/docs/openai.md)

In [29]:
from openai import OpenAI

# Initialize the OpenAI API client
client =  OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama", 
    )

chat_completion = client.chat.completions.create(
    model="llama3.2:latest",
    messages=[
        {"role": "user",
        "content": "tell me about Artificial Intelligence"},
    ]
)

In [30]:
print(type(chat_completion))
chat_completion

<class 'openai.types.chat.chat_completion.ChatCompletion'>


ChatCompletion(id='chatcmpl-689', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.\n\nThere are several types of AI:\n\n1. **Narrow or Weak AI**: This type of AI is designed to perform a specific task, such as facial recognition, sentiment analysis, or decision-making. Narrow AI has achieved state-of-the-art performance in its area of focus but does not possess general intelligence.\n\n2. **General or Strong AI**: General AI refers to intelligent machines that can understand any problem and apply it to the broader picture. Despite many efforts in this field, no algorithm can yet outperform humans across all tasks.\n\n3. **Superintelligence**: A hypotheti

In [31]:
chat_completion.choices[0].message.content

"Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.\n\nThere are several types of AI:\n\n1. **Narrow or Weak AI**: This type of AI is designed to perform a specific task, such as facial recognition, sentiment analysis, or decision-making. Narrow AI has achieved state-of-the-art performance in its area of focus but does not possess general intelligence.\n\n2. **General or Strong AI**: General AI refers to intelligent machines that can understand any problem and apply it to the broader picture. Despite many efforts in this field, no algorithm can yet outperform humans across all tasks.\n\n3. **Superintelligence**: A hypothetical AI system capable of surpassing human intelligence in its capabilities, leading to concerns about job displacement, existential ri

In [35]:
# Ensure the OpenAI client is initialized only if not already done
if not isinstance(client, OpenAI):
    client = OpenAI(
        base_url='http://localhost:11434/v1/',
        api_key='ollama',
    )

def basic_chat():
    """Basic chat completion example with improved error handling"""
    try:
        response = client.chat.completions.create(
            model="llama3.2:latest",
            messages=[
                {"role": "system", "content": "You are a helpful assistant experienced in Robotics science."},
                {"role": "user", "content": "What are the three laws of robotics?"}
            ]
        )
        print(response.choices[0].message.content)
    except Exception as e:
        print("An error occurred during the chat completion:", str(e))

basic_chat()

The Three Laws of Robotics, as developed by Isaac Asimov, are principles that guide the actions and decisions of robots in various fields, including robotics science. These laws aim to ensure safety, efficiency, and responsible behavior for both humans and robots:

1. **First Law:** A robot must not injure a human being or, through inaction, allow a human being to come to harm.

In other words, if a robot perceives that it can cause injury or harm to a human, it must take steps to prevent that from happening. This law prioritizes human safety above all else.

2. **Second Law:** A robot must obey the orders given to it by human beings except where such orders would conflict with the First Law.

This law establishes the robot's loyalty and obedience towards its human creators and operators, but also ensures that these instructions do not compromise the robot's primary responsibility: protecting humans from harm.

3. **Third Law:** A robot must protect its own existence as long as such pr

### [Ollama Python Library](https://github.com/ollama/ollama-python)

In [38]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2',
                            messages=[
                            {
                                'role': 'user',
                                'content': 'Why is the sky blue?',
                            },
                            ])

# Print the response from the chat
print(response.message.content)

The sky appears blue because of a phenomenon called scattering, which occurs when sunlight interacts with the tiny molecules of gases in the Earth's atmosphere. Here's a simplified explanation:

1. **Sunlight enters the Earth's atmosphere**: When the sun shines, it emits all colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet) as light.
2. **Light encounters tiny molecules**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
3. **Scattering occurs**: When light hits these tiny molecules, it is scattered in all directions. This scattering effect is more pronounced for shorter wavelengths (like blue and violet) because they are more easily deflected by the small molecules.
4. **Blue light is scattered more than other colors**: Due to this phenomenon, the blue light is distributed throughout the atmosphere in all directions,

#### Streaming responses

In [39]:
from ollama import chat

stream = chat(
    model='llama3.2',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
    print(chunk['message']['content'], end='', flush=True)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered that when sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen.

These gas molecules are much smaller than the wavelength of light, which means they scatter the shorter (blue) wavelengths more efficiently than the longer (red) wavelengths. This is known as Rayleigh scattering.

Here's what happens:

1. Sunlight enters Earth's atmosphere.
2. The short-wavelength blue light (around 450-495 nanometers) is scattered in all directions by the tiny gas molecules.
3. The shorter wavelength is bent (or scattered) more than the longer wavelengths, such as red and orange light (around 620-750 nanometers).
4. Our eyes perceive the scattered blue light as a bright blue color.

The amount of scattering that occurs depends on the angle of view and atmospheric conditions, like air pollution and humidity. That's why the 

#### Custom client


In [ ]:
from ollama import Client
client = Client(
    host='http://localhost:11434',
    headers={'x-some-header': 'some-value'}
)
response = client.chat(
                model='llama3.2', 
                messages=[
                        {
                            'role': 'user',
                            'content': 'Why is the sky blue?',
                        },
                        ])

In [43]:
response.message.content    

"The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.\n\nHere's what happens:\n\n1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).\n2. These gas molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.\n3. This is because the smaller molecules are more efficient at scattering the shorter wavelengths of light.\n4. As a result, the blue light is scattered in all directions and reaches our eyes from all parts of the sky, giving it a blue appearance.\n\nOn the other hand, the red light is not scattered as much and continues to travel in a straight line, reaching our eyes only when it comes from a distant object or the horizon. This is why the sky often appears more blue during the daytime and takes on hues of red or orange during sunri

#### Async client

In [ ]:
from ollama import AsyncClient
import nest_asyncio

# Allow nested event loops in Jupyter Notebook
nest_asyncio.apply()

async def chat():
    message = {'role': 'user', 
                'content': 'Why is the sky blue?'}
    response = await AsyncClient().chat(model='llama3.2', 
                                        messages=[message])
    print(response.message.content)

# Run the async function
await chat()

The sky appears blue because of a phenomenon called scattering, which occurs when sunlight interacts with the tiny molecules of gases in the Earth's atmosphere.

Here's what happens:

1. Sunlight enters Earth's atmosphere and encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. The shorter (blue) wavelengths of light are scattered more than the longer (red) wavelengths by these gas molecules.
3. This scattering effect is more pronounced for blue light because it has a shorter wavelength, which allows it to change direction more easily.
4. As the scattered blue light travels through the atmosphere, our eyes see it and perceive the sky as blue.

This phenomenon is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century. The reason why we don't see the sky as red is because red light has a longer wavelength, which is less affected by the gas molecules and therefore doesn't scatter as much.

It's

### Langchain

In [46]:
import langchain
print(langchain.__version__)

0.3.24


In [48]:
from langchain_community.llms import Ollama
ollama = Ollama(
    base_url='http://localhost:11434',
    model="llama3.2"
)
print(ollama.invoke("why is the sky blue"))

The reason why the sky appears blue is a fascinating phenomenon that has captivated humans for centuries. Here's a simplified explanation:

**Scattering of Light**

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the light in all directions. However, not all colors of light are scattered equally.

**Short-Wavelength Blues and Violets Scattered More**

The shorter, blue wavelengths are scattered more than the longer, red wavelengths. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century. The smaller the wavelength, the more easily it can be scattered by the tiny molecules of gases.

**Why Blues Appear More Than Reds**

There are a few reasons why blue light is scattered more than red light:

1. **Wavelength**: Blue light has a shorter wavelength (around 450-495 nanometers) compared to red light (around 620-750 nano

In [50]:
from langchain_community.llms import Ollama

# Initialize Ollama
ollama = Ollama(
    base_url='http://localhost:11434',
    model="llama3.2" 
)

prompt="""
write a one line description of egypt
""".strip()

print(ollama.invoke(prompt))

Egypt is an ancient and mystical land, rich in history and culture, with the iconic Pyramids of Giza, Nile River, and vibrant cities like Cairo and Alexandria.


In [ ]:
from helpers import get_settings
from openai import OpenAI
import os 

setting = get_settings()

client = OpenAI(
    api_key = setting.OPENAI_API_KEY
)

llm_model = setting.LLM_MODEL

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

from langchain_huggingface import HuggingFacePipeline

In [ ]:
# Model and tokenizer setup
model_id = env_values['MODEL_ID']

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                quantization_config=quantization_config,
                                                device_map='auto')

In [ ]:
# Set up pipeline
pipe = pipeline("text-generation",
                model=base_model,
                tokenizer=tokenizer,
                max_length=256,
                truncation=True,  # Explicitly enable truncation
                do_sample=True,
                temperature=0.6,
                top_p=0.95,
                repetition_penalty=1.2)

# Initialize LangChain HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Suggest 2 ways to lose my weight.

Answer:"""

print(llm.invoke(template))

In [ ]:
prompt_1 = """
Suggest 2 ways to lose my weight.
""".strip()

prompt_2 = """
Tell me a joke
""".strip()

llm_results = llm.generate([ prompt_1, prompt_2 ])
llm_results.generations[1][0].text

In [ ]:
# Direct pipeline usage
pipeline_output = pipe(template, return_full_text=False)
print("Pipeline Output:", pipeline_output[0]['generated_text'])